## Imports

In [2]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob

## Data Read-in

In [40]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

In [41]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Data Clean

In [42]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [43]:
df = df.dropna(subset=['SOLD DATE'])

In [44]:
# Define list of desired months (excluding current month)
desired_months = ['November']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [45]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    722
Name: count, dtype: int64
-------
$/SQUARE FEET
False    715
True       7
Name: count, dtype: int64
-------
YEAR BUILT
False    722
Name: count, dtype: int64
-------


In [46]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Miami/700-NE-26th-Ter-33137/unit-5201/home/186198982


In [47]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [48]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [49]:
df_filtered = df_filtered.drop_duplicates()

In [59]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
30,PAST SALE,November-13-2023,Condo/Co-op,15201 NE 6th Ave Unit C110,Miami,FL,33162.0,83000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15201-NE-6th-A...,MARMLS,A11456422,N,Y,25.915314,-80.186602
33,PAST SALE,November-30-2023,Condo/Co-op,20337 NE 2nd Ave #1,Miami,FL,33179.0,85000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20337-NE-2nd-A...,Beaches MLS,F10400779,N,Y,25.962503,-80.197837
0,PAST SALE,November-20-2023,Condo/Co-op,115 NE 202nd Ter Unit M20,Miami Gardens,FL,33179.0,100000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/115-NE-202nd-T...,Beaches MLS,RX-10929868,N,Y,25.959879,-80.199957
18,PAST SALE,November-14-2023,Condo/Co-op,20120 NE 2nd Ave Unit W23,Miami Gardens,FL,33179.0,107500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20120-NE-2nd-A...,MARMLS,A11441718,N,Y,25.959382,-80.201001
43,PAST SALE,November-30-2023,Condo/Co-op,2025 NE 164th St #307,North Miami Beach,FL,33162.0,112000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/20...,MARMLS,A11417486,N,Y,25.927607,-80.159386
49,PAST SALE,November-9-2023,Condo/Co-op,2025 NE 164th St #618,North Miami Beach,FL,33162.0,115000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/20...,Beaches MLS,F10325561,N,Y,25.927607,-80.159386
48,PAST SALE,November-9-2023,Condo/Co-op,2025 NE 164th St #1018,North Miami Beach,FL,33162.0,115000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/20...,Beaches MLS,F10325559,N,Y,25.927607,-80.159386
6,PAST SALE,November-14-2023,Condo/Co-op,1670 NE 191st St Unit 105-3,Miami,FL,33179.0,120000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1670-NE-191st-...,MARMLS,A11458650,N,Y,25.948852,-80.168863
47,PAST SALE,November-2-2023,Condo/Co-op,15201 NE 6th Ave Unit C112,Miami,FL,33162.0,130000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15201-NE-6th-A...,MARMLS,A11384944,N,Y,25.915314,-80.186602
23,PAST SALE,November-16-2023,Condo/Co-op,494 NW 165th Street Rd #507,Miami,FL,33169.0,138000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/494-NW-165th-S...,MARMLS,A11458150,N,Y,25.923147,-80.206728


In [51]:
print(df_filtered['URL'].iloc[36])

https://www.redfin.com/FL/Miami/444-NE-30th-St-33137/unit-507/home/43343487


In [52]:
# Correct the prices, if needed
df_filtered.at[7,'PRICE']=(250000)
df_filtered.at[36,'PRICE']=(365000)

In [ ]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [60]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE


In [61]:
# # Corrections, if needed
df_filtered.at[7,'$/SQUARE FEET']=(250000/960)
df_filtered.at[36,'$/SQUARE FEET']=(365000/652)

In [62]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
113,400000.0,1310 15th St #4,Miami Beach,94.0
30,83000.0,15201 NE 6th Ave Unit C110,Miami,112.0
6,120000.0,1670 NE 191st St Unit 105-3,Miami,113.0
37,155000.0,1750 NE 191st St Unit 111-2,Miami,118.0
0,100000.0,115 NE 202nd Ter Unit M20,Miami Gardens,120.0
33,85000.0,20337 NE 2nd Ave #1,Miami,123.0
18,107500.0,20120 NE 2nd Ave Unit W23,Miami Gardens,129.0
47,130000.0,15201 NE 6th Ave Unit C112,Miami,141.0
43,112000.0,2025 NE 164th St #307,North Miami Beach,142.0
48,115000.0,2025 NE 164th St #1018,North Miami Beach,146.0


In [33]:
print(df_filtered.URL.iloc[113])

https://www.redfin.com/FL/Miami-Beach/1310-15th-St-33139/unit-4/home/187604711


In [18]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [63]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [64]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [65]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [66]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [67]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [68]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"November 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)
        
folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [69]:
m.save('index.html')

## Summary Info

In [70]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [71]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [72]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [86]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/334-Euclid-Ave-33139/unit-303/home/43365517


In [74]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
APOGEE CONDO, 800 S Pointe Dr #1504 Miami Beach | Price $19,250,000 | $4,631 psf | Year built: 2008
Least Expensive
OLA CONDO, 15201 NE 6th Ave Unit C110 Miami | Price $83,000 | $112 psf | Year built: 1967
Highest Price Per Square Foot
APOGEE CONDO, 800 S Pointe Dr #1504 Miami Beach | Price $19,250,000 | $4,631 psf | Year built: 2008
Lowest Price Per Square Foot
ALTON BEACH BAY FRONT RE-, 1310 15th St #4 Miami Beach | Price $400,000 | $94 psf | Year built: 1947
Newest
Estates at Acqualina, 17975 Collins Ave #2001 Sunny Isles Beach | Price $11,950,000 | $nan psf | Year built: 2023
Oldest
THE FOUNTAIN CONDO HOTEL, 334 Euclid Ave #303 Miami Beach | Price $420,000 | $972 psf | Year built: 1924


## Time on Market Calculator

In [31]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [79]:
from datetime import datetime, timedelta

date1 = datetime(2023, 10, 19) ## List (Earlier) date
date2 = datetime(2023, 11, 13) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

25


## Map URL Snagger

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [34]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_oct_2023


## Get Summary Data

In [75]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 672
--------
Total sale price: $504,379,726
--------
Median sale price: $400,000
--------
Max sale price: $19,250,000
--------
Min sale price: $83,000
------------------------------------------------
PSF INFO
Max price per square foot: $4,631
--------
Min price per square foot: $94
--------
Median price per square foot: $413
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1924.0
----------
Average building age: 1988.2113095238096
